# VintedOS RAG System Demonstration

**Professional showcase of the VintedOS intelligent query system**

This notebook demonstrates the complete RAG (Retrieval-Augmented Generation) agent capabilities on a comprehensive demo database with 15 transactions, 33 items, and full relationship schema.

## What This Demonstrates

✅ **Natural Language Queries** - Ask questions in plain English  
✅ **Intelligent Routing** - Automatically selects the right tools  
✅ **Multi-turn Conversations** - Maintains context across queries  
✅ **Database Analytics** - Revenue, statistics, and insights  
✅ **Error Handling** - Gracefully handles edge cases  

## Database Contents

- **15 Transactions** covering all status types (PENDING, PARSED, MATCHED, PRINTED, COMPLETED, FAILED)
- **33 Items** ranging from £3.50 to £285.00
- **3 Pipeline Runs** simulating real execution history
- **Full Schema**: Gmail messages, attachments, print jobs, processing logs

---

## Setup & Initialization

In [ ]:
# Initialize demo mode - CRITICAL: Must be done before imports
import sys
from pathlib import Path

# Add project to path
project_root = Path.cwd()
sys.path.insert(0, str(project_root))

# Set demo mode FIRST
from src.agent.tools import set_demo_mode
set_demo_mode(True)

print("✓ Demo mode enabled")
print("✓ Using demo_db.db (safe testing environment)")

In [ ]:
# Import agent components
from src.agent.core.llm_client import LLMClient
from src.agent.core.memory import ConversationMemory
from src.agent.core.react_loop import ReActLoop
from src.agent.tools.query_tools import ALL_QUERY_TOOLS
from src.services.query_service import QueryService

print(f"✓ Loaded {len(ALL_QUERY_TOOLS)} query tools")
print("✓ Agent components imported")

In [ ]:
# Verify demo database
from sqlmodel import Session, select, func
from src.services.database import DatabaseService
from src.models import Transaction, Item, PipelineRun

db = DatabaseService(demo_mode=True)

with Session(db.engine) as session:
    tx_count = session.exec(select(func.count(Transaction.id))).one()
    item_count = session.exec(select(func.count(Item.id))).one()
    run_count = session.exec(select(func.count(PipelineRun.id))).one()
    
    print("📊 Demo Database Statistics:")
    print(f"   • {tx_count} Transactions")
    print(f"   • {item_count} Items")
    print(f"   • {run_count} Pipeline Runs")
    print("\n✅ Demo database verified and ready!")

In [ ]:
# Initialize RAG agent
llm_client = LLMClient(
    provider="google",
    model="gemini-1.5-flash",
    temperature=0.1
)

memory = ConversationMemory(max_messages=20)

agent = ReActLoop(
    llm_client=llm_client,
    tools=ALL_QUERY_TOOLS,
    memory=memory,
    max_iterations=10
)

print("🤖 RAG Agent initialized and ready!")
print(f"   Model: {llm_client.model}")
print(f"   Tools: {len(ALL_QUERY_TOOLS)} available")

---
## Demo 1: Basic Transaction Queries

Demonstrate simple, single-turn queries for transaction data.

In [ ]:
# Query 1: Recent transactions
result = agent.run("Show me the 5 most recent transactions", use_memory=False)

print("\n" + "="*60)
print("Query: Show me the 5 most recent transactions")
print("="*60)
print(result.answer)
print(f"\n📊 Steps taken: {result.total_steps}")

In [ ]:
# Query 2: Specific transaction details
result = agent.run("What are the details of transaction 9876543210?", use_memory=False)

print("\n" + "="*60)
print("Query: What are the details of transaction 9876543210?")
print("="*60)
print(result.answer)
print(f"\n📊 Steps taken: {result.total_steps}")

In [ ]:
# Query 3: Failed transactions
result = agent.run("Are there any failed transactions?", use_memory=False)

print("\n" + "="*60)
print("Query: Are there any failed transactions?")
print("="*60)
print(result.answer)
print(f"\n📊 Steps taken: {result.total_steps}")

---
## Demo 2: Analytics & Revenue Queries

Showcase the system's ability to perform business analytics.

In [ ]:
# Query 4: Revenue analysis
result = agent.run("What's the total revenue from completed transactions?", use_memory=False)

print("\n" + "="*60)
print("Query: What's the total revenue from completed transactions?")
print("="*60)
print(result.answer)
print(f"\n📊 Steps taken: {result.total_steps}")

In [ ]:
# Query 5: Dashboard summary
result = agent.run("Give me a complete dashboard summary", use_memory=False)

print("\n" + "="*60)
print("Query: Give me a complete dashboard summary")
print("="*60)
print(result.answer)
print(f"\n📊 Steps taken: {result.total_steps}")

---
## Demo 3: Complex Multi-Turn Conversation

Demonstrate contextual awareness across multiple related queries.

In [ ]:
# Reset memory for clean conversation
memory.clear()

# Turn 1: Initial query
result1 = agent.run("Show me transactions with more than 3 items", use_memory=True)

print("\n" + "="*60)
print("👤 User: Show me transactions with more than 3 items")
print("="*60)
print(f"🤖 Agent: {result1.answer}")

In [ ]:
# Turn 2: Follow-up using context
result2 = agent.run("What's the total value of those orders?", use_memory=True)

print("\n" + "="*60)
print("👤 User: What's the total value of those orders?")
print("="*60)
print(f"🤖 Agent: {result2.answer}")

In [ ]:
# Turn 3: Another follow-up
result3 = agent.run("Which customer spent the most?", use_memory=True)

print("\n" + "="*60)
print("👤 User: Which customer spent the most?")
print("="*60)
print(f"🤖 Agent: {result3.answer}")

print("\n💬 Conversation preserved across 3 turns!")

---
## Demo 4: Edge Cases & Error Handling

Show robust handling of various query types.

In [ ]:
# Query 6: Non-existent transaction
result = agent.run("Show me transaction 9999999999", use_memory=False)

print("\n" + "="*60)
print("Query: Show me transaction 9999999999 (non-existent)")
print("="*60)
print(result.answer)
print(f"\n📊 Gracefully handled: {result.success}")

In [ ]:
# Query 7: Ambiguous query
result = agent.run("What's the status of my order?", use_memory=False)

print("\n" + "="*60)
print("Query: What's the status of my order? (ambiguous)")
print("="*60)
print(result.answer)
print(f"\n📊 Handled ambiguity: {result.success}")

---
## Demo 5: Advanced Analytics

Demonstrate complex data analysis capabilities.

In [ ]:
# Query 8: Print job success rate
result = agent.run("What's the print job success rate?", use_memory=False)

print("\n" + "="*60)
print("Query: What's the print job success rate?")
print("="*60)
print(result.answer)
print(f"\n📊 Steps taken: {result.total_steps}")

In [ ]:
# Query 9: Status breakdown
result = agent.run("Give me a breakdown of transaction statuses", use_memory=False)

print("\n" + "="*60)
print("Query: Give me a breakdown of transaction statuses")
print("="*60)
print(result.answer)
print(f"\n📊 Steps taken: {result.total_steps}")

---
## Performance Summary

In [ ]:
print("="*60)
print("RAG SYSTEM DEMONSTRATION COMPLETE")
print("="*60)
print()
print("✅ Capabilities Demonstrated:")
print("   • Natural language query understanding")
print("   • Automatic tool selection and execution")
print("   • Multi-turn contextual conversations")
print("   • Complex analytics and aggregations")
print("   • Robust error handling")
print("   • Business intelligence insights")
print()
print("📊 Demo Database:")
print("   • 15 transactions (all status types)")
print("   • 33 items (£3.50 - £285.00)")
print("   • Full schema with relationships")
print("   • Production-identical structure")
print()
print("🚀 Next Steps:")
print("   • Interactive CLI: python agent.py --demo")
print("   • Run ETL pipeline: python run.py --demo")
print("   • See DEMO_MODE.md for full documentation")
print()
print("="*60)